# Limpieza de datos con PySpark: Data Science Job Posting on Glassdoor

Los [datos](https://tajamar365.sharepoint.com/:x:/s/3405-MasterIA2024-2025/ETYTQ0c-i6FLjM8rZ4iT1cgB6ipFAkainM-4V9M8DXsBiA?e=PeMtvh) fueron extraídos (scrapeados) del sitio web de Glassdoor y recoge los salarios de distintos puestos relacionados a Data.

### Resolver los siguientes requerimientos, para cada operación/moficación imprima como van quedadndo los cambios.

filepath = 

1. Cargar los datos y mostrar el esquema o la informacion de las columnas y el tip de dato de cada columna

In [0]:
from pyspark.sql import (
    SparkSession,
    types,
    functions as F
)

spark = (
    SparkSession
    .builder 
    .appName("Job posting")
    .getOrCreate()
)

In [0]:
df = spark.read.option("header", "true") \
    .option("delimiter", ";") \
    .option("multiline", "true") \
    .option("quote", "\"") \
    .option("escape", "\"") \
    .csv("/FileStore/Examen/Caso_4/ds_jobs.csv")

In [0]:
df.display()

In [0]:
df.printSchema()

2. Eliminar duplicados

In [0]:
# Importar las funciones necesarias
from pyspark.sql.functions import col, count

# Agrupar por todas las columnas y contar las ocurrencias
df_duplicates = df.groupBy(df.columns).agg(count("*").alias("count"))

# Filtrar solo los duplicados
df_duplicates = df_duplicates.filter(col("count") > 1)

# Mostrar los duplicados
df_duplicates.display()

In [0]:
# Eliminar los registros duplicados basados en todas las columnas, aunque en este caso no se encuentra ninguno
df_clean = df.dropDuplicates()

# Mostrar el DataFrame limpio
df_clean.display()

3. Decidir que hacer con los datos faltantes 

4. Decidir que hacer con los valores nulos

Eliminar los datos nulos

In [0]:
# Eliminar las filas con valores nulos directamente en el DataFrame original
df = df.dropna()

# Verificar el resultado
df.display()

5. ¿Cuántos registros tiene el csv?

In [0]:
# Contar el número de registros en el DataFrame
num_registros = df.count()

# Mostrar el resultado
print(f"El número total de registros es: {num_registros}")

6. Mostrar los valores únicos de `Job title` 

In [0]:
# Obtener los valores únicos de la columna "Job Title"
valores_unicos = df.select("Job Title").distinct()

# Mostrar los valores únicos
valores_unicos.display()  # truncate=False para ver los valores completos

7. Remover la letra `K` de la columna `Salary Estimate` y multiplicar por 1000.

In [0]:
from pyspark.sql.functions import regexp_replace, col

df = df.withColumn(
    "Salary Estimate",
    (regexp_replace(col("Salary Estimate"), "K", "000"))
)

df.display()

8. Mostrar los valores únicos del campo `Salary Estimate`

In [0]:
# Obtener los valores únicos de la columna "Salary Estimate"
valores_unicos_salario = df.select("Salary Estimate").distinct()

# Mostrar los valores únicos
valores_unicos_salario.display()

9. Eliminar `(Glassdoor est.)` y `(Employer est.)` del campo `Salary Estimate`

In [0]:
from pyspark.sql.functions import regexp_replace

# Eliminar "(Glassdoor est.)" y "(Employer est.)" de la columna "Salary Estimate"
df = df.withColumn(
    "Salary Estimate",
    regexp_replace(col("Salary Estimate"), "\\(Glassdoor est.\\)", "")  # Eliminar "(Glassdoor est.)"
)

df = df.withColumn(
    "Salary Estimate",
    regexp_replace(col("Salary Estimate"), "\\(Employer est.\\)", "")  # Eliminar "(Employer est.)"
)

# Mostrar los resultados para verificar
df.select("Salary Estimate").show(truncate=False)

10. Mostrar de mayor a menor los valores del campo `Salary Estimate`

In [0]:
# Ordenar por el campo Salary Estimate de forma descendente
df.orderBy("Salary Estimate", ascending=False).select("Salary Estimate").show(truncate=False)

11. De la columna `Job Description` quitar los saltos de linea `\n` del texto

In [0]:
from pyspark.sql.functions import regexp_replace

# Reemplazar los saltos de línea (\n) en la columna "Job Description"
df = df.withColumn(
    "Job Description",
    regexp_replace("Job Description", r"\n", " ")
)

# Mostrar los resultados para verificar
df.select("Job Description").display()

12. De la columna `Rating` muestre los valores unicos.

In [0]:
# Obtener los valores únicos de la columna "Rating"
valores_unicos_rating = df.select("Rating").distinct()

# Mostrar los valores únicos
valores_unicos_rating.display()

13. Del campo `Rating` reemplazar los `-1.0` por `0.0`.

In [0]:
from pyspark.sql.functions import when, col

# Reemplazar -1.0 por 0.0 en la columna "Rating"
df = df.withColumn(
    "Rating",
    when(col("Rating") == -1.0, 0.0).otherwise(col("Rating"))
)

# Mostrar los resultados para verificar
df.select("Rating").display()

In [0]:
# Filtrar las filas donde Rating es 0.0
df_zero_rating = df.filter(col("Rating") == 0.0)

# Mostrar los resultados
df_zero_rating.display()

14. Mostrar los valores unicos y ordenar los valores del campo `Company Name`.

In [0]:
# Obtener los valores únicos de la columna "Company Name" y ordenarlos alfabéticamente
df_unique_companies = df.select("Company Name").distinct().orderBy("Company Name")

# Mostrar los valores únicos ordenados
df_unique_companies.display()

15. Quitar todos los caracteres innecesarios que encuentres en el campo `Company Name`. Por ejemplo los saltos de linea `\n`

In [0]:
from pyspark.sql.functions import regexp_replace

# Eliminar saltos de línea (\n) y otros caracteres innecesarios en "Company Name"
df = df.withColumn(
    "Company Name",
    regexp_replace("Company Name", r"\n", " ")  # Reemplazar saltos de línea por un espacio
)

# También puedes reemplazar otros caracteres no deseados, como espacios extra:
df = df.withColumn(
    "Company Name",
    regexp_replace("Company Name", r"\s+", " ").trim() 
)

# Mostrar los resultados
df.select("Company Name").display()

16. En el campo `Location` convertir esa columna en dos: `City` y `State`. Las ciudades que tengas en `Location` asignar a la columna `City`. Lo mismo para `State`. Luego elimine la columna `Location`.

In [0]:
from pyspark.sql.functions import split, col

# Dividir la columna "Location" en dos columnas: "City" y "State"
df = df.withColumn("City", split(col("Location"), ",")[0]) \
       .withColumn("State", split(col("Location"), ",")[1])

# Eliminar la columna "Location"
df = df.drop("Location")

# Mostrar los resultados para verificar
df.select("City", "State").show(truncate=False)

17. Repetir la misma lógica de la pregunta 16 pero para el campo `Headquarters`. En Headquarters dejar solo la ciudad, mientras que para el estado añadirla a una columna nueva ` Headquarter State`.

In [0]:
from pyspark.sql.functions import split, col

# Dividir la columna "Headquarters" en dos columnas: "City" y "Headquarter State"
df = df.withColumn("City", split(col("Headquarters"), ",")[0]) \
       .withColumn("Headquarter State", split(col("Headquarters"), ",")[1])

# Eliminar la columna "Headquarters"
df = df.drop("Headquarters")

# Mostrar los resultados para verificar
df.select("City", "Headquarter State").show(truncate=False)

18. Muestre los valores únicos del campo `Headquarter State` 

In [0]:
# Obtener los valores únicos de la columna "Headquarter State"
valores_unicos_headquarter_state = df.select("Headquarter State").distinct()

# Mostrar los valores únicos
valores_unicos_headquarter_state.show(truncate=False)

19. Mostrar valores unicos del campo `Size`.

In [0]:
# Obtener los valores únicos de la columna "Size"
valores_unicos_size = df.select("Size").distinct()

# Mostrar los valores únicos
valores_unicos_size.display()

20. Quitar 'employee' de los registros del campo `Size`. Elimine tambien otros caracteres basura.

21. Reemplazar la palabra 'to' por '-' en todos los registros del campo `Size`. Reemplazar tambien '-1' por 'Unknown'. 

In [0]:
from pyspark.sql.functions import regexp_replace

# Reemplazar 'to' por '-' y '-1' por 'Unknown' en la columna "Size"
df = df.withColumn(
    "Size",
    regexp_replace(regexp_replace("Size", "to", "-"), "-1", "Unknown")
)

# Mostrar los resultados para verificar
df.select("Size").show(truncate=False)

22. Mostrar el tipo de dato del campo `Type of ownership` y sus registros unicos.

In [0]:
# Mostrar el tipo de dato de la columna "Type of ownership"
tipo_dato = df.select("Type of ownership").dtypes[0][1]
print(f"Tipo de dato de 'Type of ownership': {tipo_dato}")

# Obtener los valores únicos de la columna "Type of ownership"
valores_unicos_type_of_ownership = df.select("Type of ownership").distinct()

# Mostrar los valores únicos
valores_unicos_type_of_ownership.show(truncate=False)

23. Cambiar '-1' por 'Unknown' en todos los registros del campo `Type of ownership`.

In [0]:
from pyspark.sql.functions import regexp_replace

# Reemplazar '-1' por 'Unknown' en la columna "Type of ownership"
df = df.withColumn(
    "Type of ownership",
    regexp_replace("Type of ownership", "-1", "Unknown")
)

# Mostrar los resultados para verificar
df.select("Type of ownership").show(truncate=False)

24. Cambiar:  
-  `Company - Public` por `Public Company`  
-  `Company - Private` por `Private Company`  
-  `Private Practice / Firm` por `Private Company`  
-  `Subsidiary or Business Segment` por `Business`  
-  `College / University` por `Education`  
En todos los registros del campo `Type of ownership`.

In [0]:
from pyspark.sql.functions import regexp_replace

# Reemplazar los valores en la columna "Type of ownership"
df = df.withColumn(
    "Type of ownership",
    regexp_replace("Type of ownership", "Company - Public", "Public Company")
)

df = df.withColumn(
    "Type of ownership",
    regexp_replace("Type of ownership", "Company - Private", "Private Company")
)

df = df.withColumn(
    "Type of ownership",
    regexp_replace("Type of ownership", "Private Practice / Firm", "Private Company")
)

df = df.withColumn(
    "Type of ownership",
    regexp_replace("Type of ownership", "Subsidiary or Business Segment", "Business")
)

df = df.withColumn(
    "Type of ownership",
    regexp_replace("Type of ownership", "College / University", "Education")
)

# Mostrar los resultados para verificar
df.select("Type of ownership").show(truncate=False)

25. Mostrar el tipo de dato y los valores unicos del campo `Industry`.

In [0]:
# Mostrar el tipo de dato de la columna "Industry"
tipo_dato_industry = df.select("Industry").dtypes[0][1]
print(f"Tipo de dato de 'Industry': {tipo_dato_industry}")

# Obtener los valores únicos de la columna "Industry"
valores_unicos_industry = df.select("Industry").distinct()

# Mostrar los valores únicos
valores_unicos_industry.show(truncate=False)

26. En el mismo campo de `Industry` reemplazar '-1' por 'Not Available' y '&' por 'and'.  Vuelva a imprimir los valores unicos en orden alfabético.

In [0]:
from pyspark.sql.functions import regexp_replace

# Reemplazar '-1' por 'Not Available' y '&' por 'and' en la columna "Industry"
df = df.withColumn(
    "Industry",
    regexp_replace(regexp_replace("Industry", "-1", "Not Available"), "&", "and")
)

# Obtener los valores únicos de la columna "Industry" en orden alfabético
valores_unicos_industry = df.select("Industry").distinct().orderBy("Industry")

# Mostrar los valores únicos en orden alfabético
valores_unicos_industry.show(truncate=False)

27. Para el campo `Sector`, muestre el tipo de dato y los valores únicos.

In [0]:
# Mostrar el tipo de dato de la columna "Sector"
tipo_dato_sector = df.select("Sector").dtypes[0][1]
print(f"Tipo de dato de 'Sector': {tipo_dato_sector}")

# Obtener los valores únicos de la columna "Sector"
valores_unicos_sector = df.select("Sector").distinct()

# Mostrar los valores únicos
valores_unicos_sector.show(truncate=False)

28. Aplica la misma lógica de la pregunta 26 pero sobre el campo `Sector`.

In [0]:
from pyspark.sql.functions import regexp_replace

# Reemplazar '-1' por 'Not Available' y '&' por 'and' en la columna "Sector"
df = df.withColumn(
    "Sector",
    regexp_replace(regexp_replace("Sector", "-1", "Not Available"), "&", "and")
)

# Obtener los valores únicos de la columna "Sector" en orden alfabético
valores_unicos_sector = df.select("Sector").distinct().orderBy("Sector")

# Mostrar los valores únicos en orden alfabético
valores_unicos_sector.show(truncate=False)

29. Para el campo `Revenue`, muestre el tipo de dato y los valores únicos en orden ascedente.

In [0]:
# Mostrar el tipo de dato de la columna "Revenue"
tipo_dato_revenue = df.select("Revenue").dtypes[0][1]
print(f"Tipo de dato de 'Revenue': {tipo_dato_revenue}")

# Obtener los valores únicos de la columna "Revenue" en orden ascendente
valores_unicos_revenue = df.select("Revenue").distinct().orderBy("Revenue")

# Mostrar los valores únicos en orden ascendente
valores_unicos_revenue.show(truncate=False)

30. En el campo `Revenue`, cambiar:  
-  `-1` por `N/A`  
-  `Unknown / Non-Applicable` por `N/A`  
-  `Less than $1 million (USD)` por `Less than 1`
-  Quitar `$` y `(USD)`

In [0]:
from pyspark.sql.functions import regexp_replace

# Reemplazar '-1' por 'N/A', 'Unknown / Non-Applicable' por 'N/A', 
# 'Less than $1 million (USD)' por 'Less than 1' y quitar '$' y '(USD)'.
df = df.withColumn(
    "Revenue",
    regexp_replace("Revenue", "-1", "N/A")
)

df = df.withColumn(
    "Revenue",
    regexp_replace("Revenue", "Unknown / Non-Applicable", "N/A")
)

df = df.withColumn(
    "Revenue",
    regexp_replace("Revenue", "Less than \\$1 million \\(USD\\)", "Less than 1")
)

df = df.withColumn(
    "Revenue",
    regexp_replace("Revenue", "\\$", "")  # Eliminar el símbolo "$"
)

df = df.withColumn(
    "Revenue",
    regexp_replace("Revenue", "\\(USD\\)", "")  # Eliminar "(USD)"
)

# Mostrar los resultados para verificar
df.select("Revenue").show(truncate=False)

31. Borrar el campo `Competitors`.

In [0]:
# Eliminar la columna "Competitors"
df = df.drop("Competitors")

# Mostrar el esquema para verificar que la columna fue eliminada
df.printSchema()

In [0]:
df.display()

32. Crear tres columnas: `min_salary` (salario mínimo), `max_salary` (salario maximo) y `avg_salary` (salario promedio) a partir de los datos del campo `Salary Estimate`.

In [0]:
df = df.withColumn("SalarioSinDollar", regexp_replace(col("Salary Estimate"), "\$", ""))

df = df.withColumn(
    "min_salary", 
    split(col("SalarioSinDollar"), "-")[0].cast("int")  # Tomamos el primer valor como salario mínimo
)

df = df.withColumn(
    "max_salary", 
    split(col("SalarioSinDollar"), "-")[1].cast("int")  # Tomamos el segundo valor como salario máximo
)

df = df.withColumn(
    "avg_salary", 
    (col("min_salary") + col("max_salary")) / 2
)

df.select("index", "Job Title", "min_salary", "max_salary", "avg_salary").orderBy(col('index').asc()).display()

33. Mostrar los valores unicos del campo `Founded` y el tipo de dato.

34. Reemplazar '-1' por '2024' en todos los registros del campo `Founded`.

35. Crear una nueva columna o campo que se llame `company_age` con los datos que se deducen del campo `Founded`.

36. Crear una columna o campo que se llame: `Job Type` y en cada registro debe ir Senior, Junior o NA según los datos del campo `Job Title`.  
- Cambiar 'sr' o 'senior' o 'lead' o 'principal' por `Senior` en el campo `Job Type`. No olvidar las mayúsculas.
- Cambiar 'jr' o 'jr.' o cualquier otra variante por `Junior`.  
- En cualquier otro caso distinto a los anteriores añadir NA.

37. Muestra los registros únicos del campo `Job Type`. 

38. Partiendo del campo `Job Description` se extraer todas o las principales skills solicitadas por las empresas, por ejemplo: Python, Spark , Big Data. Cada Skill debe ir en una nueva columna de tipo Binaria ( 0 , 1) o Booleana (True,  False) de modo que cada skill va ser una nueva columna y si esa skill es solicitada por la empresa colocar 1 sino colocar 0. Por ejemplo:  

Por ejemplo:  
| Job Title         | Salary Estimate | Job Description                                 | Rating | Company Name       | Size       | Founded | Type of ownership         | Industry                       | Sector                         | Same State      | company_age | Python | Excel |
|--------------------|-----------------|-------------------------------------------------|--------|--------------------|------------|---------|---------------------------|--------------------------------|--------------------------------|----------------|-------------|--------|-------|
| Sr Data Scientist | 137000-171000   | Description The Senior Data Scientist is resp... | 3.1    | Healthfirst        | 1001-5000  | 1993    | Nonprofit Organization    | Insurance Carriers            | Insurance Carriers            | Same State      | 31          | 0      | 0     |
| Data Scientist    | 137000-171000   | Secure our Nation, Ignite your Future Join th... | 4.2    | ManTech            | 5001-10000 | 1968    | Public Company            | Research and Development      | Research and Development      | Same State      | 56          | 0      | 0     |
| Data Scientist    | 137000-171000   | Overview Analysis Group is one of the larges... | 3.8    | Analysis Group      | 1001-5000  | 1981    | Private Company           | Consulting                    | Consulting                    | Same State      | 43          | 1      | 1     |
| Data Scientist    | 137000-171000   | JOB DESCRIPTION: Do you have a passion for Da... | 3.5    | INFICON            | 501-1000   | 2000    | Public Company            | Electrical and Electronic Manufacturing | Electrical and Electronic Manufacturing | Different State | 24          | 1      | 1     |


39. Exportar dataset final a csv

40. Extraer todos los insights posibles que sean de valor o utilidad. Cree nuevas columnas, agrupar,  filtrar hacer varios plots que muestren dichos insights que sean de utilidad para una empresa o para un usuario. Elabore conclusiones con los insights encontrados. 